<a href="https://colab.research.google.com/github/Keoni808/NFL_Data_Cleaning/blob/main/NFL_Plays_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting and Imports

In [1]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Used to access personal google cloud services
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
# imports

# Data manipulation
import pandas as pd

# Regular expressions
import re

# Grab data from database
from google.cloud import bigquery

# Loading Data


In [4]:
# Client connect to bigquery project
client = bigquery.Client('nfl-data-430702')

## All Plays 2023

In [5]:
# nfl_2023_plays_query = """
#                        SELECT *
#                        FROM `nfl-data-430702.NFL_Scores.NFL-Plays-2023`
#                        """

# # Run the query, and return a pandas DataFrame
# dry_run_config = bigquery.QueryJobConfig(dry_run=True)
# dry_run_query = client.query(nfl_2023_plays_query, job_config=dry_run_config)
# print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

# safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
# safe_config_query = client.query(nfl_2023_plays_query, job_config=safe_config)

# # df_nfl_scores_data = safe_config_query.to_dataframe()

In [6]:
# df_2023_plays = safe_config_query.to_dataframe()

In [7]:
# df_2023_plays.head()

##Super Bowl Plays 2023

In [8]:
nfl_2023_sb_plays_query = """
                          SELECT *
                          FROM `nfl-data-430702.NFL_Scores.NFL-Plays-SuperBowl-2023`
                          """

# Run the query, and return a pandas DataFrame
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(nfl_2023_sb_plays_query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(nfl_2023_sb_plays_query, job_config=safe_config)

# df_nfl_scores_data = safe_config_query.to_dataframe()

This query will process 41291 bytes.


In [9]:
df_2023_plays_sb = safe_config_query.to_dataframe()

In [10]:
df_2023_plays_sb.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
0,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,1,0,-3 Yard Run,(12:15) (Shotgun) I.Pacheco left guard to KC 2...,1st & 10 at KC 27
1,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,2,0,1 Yard Pass,(11:39) (Shotgun) P.Mahomes pass short left to...,2nd & 13 at KC 24
2,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,3,0,8 Yard Pass,(11:04) (Shotgun) P.Mahomes pass short right t...,3rd & 12 at KC 25
3,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,4,0,Punt,"(10:24) T.Townsend punts 43 yards to SF 24, Ce...",4th & 4 at KC 33
4,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,4,KC,0,1,0,10 Yard Run,(6:28) (Shotgun) I.Pacheco right guard to KC 2...,1st & 10 at KC 11


In [11]:
df_2023_plays_sb.shape

(191, 15)

# Categorize Plays
- The goal here is to parse out the different values for 'PlayOutcome'
  - separate pass / run / kickoff / etc.

## Parsing Plays


In [12]:
# To see all possible play outcomes of the game.
# - From here we can categorize and clean plays accordingly
df_2023_plays_sb['PlayOutcome'].unique()

array(['-3 Yard Run', '1 Yard Pass', '8 Yard Pass', 'Punt', '10 Yard Run',
       '-4 Yard Sack', 'Pass for No Gain', '4 Yard Run', 'Kickoff',
       '7 Yard Pass', '2 Yard Run', '5 Yard Run', '52 Yard Pass',
       'Fumble', 'Pass Incomplete', '3 Yard Run', 'Run for No Gain',
       '10 Yard Pass', 'Interception', '9 Yard Pass', '5 Yard Pass',
       '6 Yard Run', '18 Yard Pass', '11 Yard Pass', '11 Yard Run',
       '-2 Yard Run', '-5 Yard Penalty', '-10 Yard Penalty',
       '12 Yard Pass', '24 Yard Run', '1 Yard Run', 'Sack',
       '-8 Yard Pass', '-1 Yard Run', '8 Yard Run', '6 Yard Pass',
       '21 Yard Pass', '3 Yard Pass', '-1 Yard Sack', 'Field Goal',
       '22 Yard Run', '2 Yard Pass', 'Touchdown Chiefs',
       'Extra Point Good', '16 Yard Pass', '13 Yard Pass', '25 Yard Pass',
       '9 Yard Run', '-3 Yard Sack', '22 Yard Pass', '-3 Yard Pass',
       '4 Yard Pass', '19 Yard Run', '5 Yard Penalty', '19 Yard Pass',
       '-4 Yard Run', '7 Yard Run', '16 Yard Run', 'Touch

In [13]:
# Looking at all unique play outcomes and categorizing them.
# - This type of approach does not feel very flexable because a play outcome can
#   arise that has not been seen yet.
df_2023_pass_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Pass')]
df_2023_run_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Run')]
df_2023_punt_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Punt')]
df_2023_sack_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Sack')]
df_2023_kickoff_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Kickoff')]
df_2023_fumble_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Fumble')]
df_2023_interception_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Interception')]
df_2023_penalty_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Penalty')]
df_2023_fieldgoal_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Field Goal')]
df_2023_touchdown_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Touchdown')]
df_2023_extrapoint_sb = df_2023_plays_sb[df_2023_plays_sb['PlayOutcome'].str.contains('Extra Point')]

plays_list = [df_2023_pass_sb,
              df_2023_run_sb,
              df_2023_punt_sb,
              df_2023_sack_sb,
              df_2023_kickoff_sb,
              df_2023_fumble_sb,
              df_2023_interception_sb,
              df_2023_penalty_sb,
              df_2023_fieldgoal_sb,
              df_2023_touchdown_sb,
              df_2023_extrapoint_sb]

## Sanity Check (All Plays Accounted for)

In [14]:
# A check to make sure that all plays have been categorized.
# - The check puts all categorized plays into a single dataframe
#   and will compare with the original dataframe to make sure
#   that they are the same.
df_check = pd.DataFrame()
for i in plays_list:
  df_check = pd.concat([df_check, i])

In [15]:
df_check = df_check.sort_index()

In [16]:
df_2023_plays_sb.equals(df_check)

True

# 'PlayDescription' Feature Breakdown

ISSUES:
- laterals?
- penalties
  - Declined or accepted?
  - What kind of penalty was it?

- I need a check to make sure that all plays have been broken down
  - Possibly add a check for each type of play that happened?

- Touchdowns
  - A passing touchdown is not included within the passing category, it is in its own.



In [17]:
####################################################
# REGULAR EXPRESSIONS USED TO LOCATE SPECIFIC DATA #
####################################################

time_on_clock_pattern = r'\(\d*:\d+\)'
formation = r'\([A-Za-z]+ ?[A-Za-z]*,? ?[A-Za-z]*\)'
name_pattern = r'\b[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\b' # Grabs all names but will only be used for Quarterback
receiver_name_pattern = r'\b [A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\b' # Receivers have a space before their name
defense_tackler_1_name_pattern = r'\([A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*' # Will have a "(" in front of the name
defense_tackler_2_name_pattern = r' [A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\)' # Will have a ")" at the end of the name
defense_pressure_name_pattern = r'\[[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\]' # Surrounded by "[]" brackets
injured_or_penalty_name = r'[A-Za-z]+-[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*' # Their team initials comes before their name (e.g. KC-B.Bob)

## Pass

### Identify Different Pass Plays
- This section is used to categorize different pass plays to see if they have to be handled differently.
  - Eventually, each category of pass play will break down into the same set of features. The question here is how does each category of pass play break down to fall into these common features?

GOAL: To create a single method that will handle each type of pass play and break them down to a common set of features.

In [18]:
df_2023_pass_sb['PlayOutcome'].unique()

array(['1 Yard Pass', '8 Yard Pass', 'Pass for No Gain', '7 Yard Pass',
       '52 Yard Pass', 'Pass Incomplete', '10 Yard Pass', '9 Yard Pass',
       '5 Yard Pass', '18 Yard Pass', '11 Yard Pass', '12 Yard Pass',
       '-8 Yard Pass', '6 Yard Pass', '21 Yard Pass', '3 Yard Pass',
       '2 Yard Pass', '16 Yard Pass', '13 Yard Pass', '25 Yard Pass',
       '22 Yard Pass', '-3 Yard Pass', '4 Yard Pass', '19 Yard Pass',
       '17 Yard Pass', '20 Yard Pass', '23 Yard Pass', '24 Yard Pass'],
      dtype=object)

In [19]:
# 3 different formats as far as I can see.
# 1. '# Yard Pass'
# 2. 'Pass Incomplete'
# 3. 'Pass for No Gain'

df_successful_passes = df_2023_pass_sb[df_2023_pass_sb['PlayOutcome'].str.contains('Yard Pass')]
df_incomplete_passes = df_2023_pass_sb[df_2023_pass_sb['PlayOutcome'].str.contains('Pass Incomplete')]
df_pass_for_no_gain = df_2023_pass_sb[df_2023_pass_sb['PlayOutcome'].str.contains('Pass for No Gain')]

### New Features & DataFrame

In [28]:
# Should add "PlayType" -> Here would be a 'pass' play type

new_columns_passing = ["PlayType", "TimeOnTheClock", "Formation", "Quarterback", "Receiver", "PassType", "PassDirection", "PassYardage",
                       "Tackler1", "Tackler2", "PressureBy",
                       "InjuriesOnPlay", "InjuredPlayers",
                       "PenaltyOnPlay", "PenaltyPlayers"]

passing_string_columns = ["PlayType", "TimeOnTheClock", "Formation", "Quarterback", "Receiver", "PassType", "PassDirection",
                          "Tackler1", "Tackler2", "PressureBy", "InjuredPlayers", "PenaltyPlayers"]

passing_int_columns = ["PassYardage"]

df_2023_pass_sb_detailed = df_2023_pass_sb.copy()
df_2023_pass_sb_detailed = df_2023_pass_sb_detailed.reindex(columns=df_2023_pass_sb_detailed.columns.tolist() + new_columns_passing)
df_2023_pass_sb_detailed[passing_string_columns] = df_2023_pass_sb_detailed[passing_string_columns].astype(str)
df_2023_pass_sb_detailed[passing_int_columns] = df_2023_pass_sb_detailed[passing_int_columns].astype(float)

In [29]:
for idx, value in df_2023_pass_sb['PlayOutcome'].items():
  play = df_2023_plays_sb['PlayDescription'].iloc[idx]

  ################
  # Play details #
  ################

  # Play Type
  df_2023_pass_sb_detailed.loc[idx, 'PlayType'] = 'Pass'

  # TimeOnTheClock
  TimeOnTheClock = re.findall(time_on_clock_pattern, play)
  df_2023_pass_sb_detailed.loc[idx, 'TimeOnTheClock'] = TimeOnTheClock[0][1:-1]

  #############
  #  OFFENSE  #
  #############

  # Formation
  Formation = re.findall(formation, play)
  if len(Formation) > 0:
    df_2023_pass_sb_detailed.loc[idx, 'Formation'] = Formation[0][1:-1]
  # Quarterback & Receiver
  Quarterback = re.findall(name_pattern, play)
  df_2023_pass_sb_detailed.loc[idx, 'Quarterback'] = Quarterback[0] # Quarterback
  Receiver = re.findall(receiver_name_pattern, play)
  if len(Receiver) > 0:
    df_2023_pass_sb_detailed.loc[idx, 'Receiver'] = Receiver[0][1:] # Receiver

  # Deep or short pass
  if value.find('Incomplete') != -1:
    df_2023_pass_sb_detailed.loc[idx, 'PassType'] = 'Incomplete'
    df_2023_pass_sb_detailed.loc[idx, 'PassYardage'] = 0
  elif value.find('No Gain') != -1:
    if play.find('short') != -1:
      df_2023_pass_sb_detailed.loc[idx, 'PassType'] = 'Short'
    elif play.find('deep') != -1:
      df_2023_pass_sb_detailed.loc[idx, 'PassType'] = 'Deep'
    df_2023_pass_sb_detailed.loc[idx, 'PassYardage'] = 0
  else:
    if int(value.split()[0]) >= 20:
      df_2023_pass_sb_detailed.loc[idx, 'PassType'] = 'Deep'
    else:
      df_2023_pass_sb_detailed.loc[idx, 'PassType'] = 'Short'
    # Yardage gained on play
    df_2023_pass_sb_detailed.loc[idx, 'PassYardage'] = int(value.split()[0])

  # Pass Direction
  if play.find('left') != -1:
    df_2023_pass_sb_detailed.loc[idx, 'PassDirection'] = 'Left'
  elif play.find('right') != -1:
    df_2023_pass_sb_detailed.loc[idx, 'PassDirection'] = 'Right'
  elif play.find('middle') != -1:
    df_2023_pass_sb_detailed.loc[idx, 'PassDirection'] = 'Middle'
  # # Yardage gained on play
  # df_successful_passes_detailed.loc[idx, 'PassYardage'] = int(value.split()[0])

  #############
  #  DEFENSE  #
  #############

  tackler_1 = re.findall(defense_tackler_1_name_pattern, play) # tackler #1 (Could be solo or the one who initiated the hit)
  if len(tackler_1) > 0:
    df_2023_pass_sb_detailed.loc[idx, 'Tackler1'] = tackler_1[0][1:]
  tackler_2 = re.findall(defense_tackler_2_name_pattern, play) # tackler #2 (equally contributed or assisted with tackle)
  if len(tackler_2) > 0:
    df_2023_pass_sb_detailed.loc[idx, 'Tackler2'] = tackler_2[0][1:-1]
  pressure = re.findall(defense_pressure_name_pattern, play)   # Player who applied pressure to passer
  if len(pressure) > 0:
    df_2023_pass_sb_detailed.loc[idx, 'PressureBy'] = pressure[0][1:-1]

  #############
  #  PENALTY  #
  #############

  if play.find('Penalty') != -1:
    penalty_name = re.findall(injured_or_penalty_name, play)
    df_2023_pass_sb_detailed.at[idx, 'PenaltyPlayers'] = penalty_name

  if play.find('injured') != -1:
    injured_name = re.findall(injured_or_penalty_name, play)
    df_2023_pass_sb_detailed.at[idx, 'InjuredPlayers'] = injured_name

In [30]:
df_2023_pass_sb_detailed

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,...,PassType,PassDirection,PassYardage,Tackler1,Tackler2,PressureBy,InjuriesOnPlay,InjuredPlayers,PenaltyOnPlay,PenaltyPlayers
1,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,...,Short,Left,1.0,C.Young,D.Greenlaw,nan,NaN,nan,NaN,nan
2,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,...,Short,Right,8.0,F.Warner,D.Greenlaw,nan,NaN,nan,NaN,nan
6,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,4,KC,0,...,Short,Left,0.0,F.Warner,nan,nan,NaN,nan,NaN,nan
10,2023,Super Bowl,SUN,02/11,49ers,Chiefs,2ND QUARTER,2,KC,0,...,Short,Left,7.0,T.Gipson,nan,nan,NaN,nan,NaN,nan
13,2023,Super Bowl,SUN,02/11,49ers,Chiefs,2ND QUARTER,2,KC,0,...,Deep,Right,52.0,J.Brown,nan,nan,NaN,nan,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,2023,Super Bowl,SUN,02/11,49ers,Chiefs,OVERTIME,1,SF,1,...,Short,Middle,2.0,C.Jones,nan,nan,NaN,nan,NaN,nan
181,2023,Super Bowl,SUN,02/11,49ers,Chiefs,OVERTIME,1,SF,1,...,Short,Left,11.0,M.Edwards,nan,nan,NaN,nan,NaN,nan
183,2023,Super Bowl,SUN,02/11,49ers,Chiefs,OVERTIME,1,SF,1,...,Deep,Left,24.0,L.Sneed,nan,G.Karlaftis,NaN,nan,NaN,nan
186,2023,Super Bowl,SUN,02/11,49ers,Chiefs,OVERTIME,1,SF,1,...,Short,Right,13.0,nan,nan,nan,NaN,nan,NaN,nan


In [31]:
df_2023_pass_sb_detailed[["PlayDescription", "PlayType", "TimeOnTheClock", "Formation", "Quarterback", "Receiver", "PassType", "PassDirection", "PassYardage",
                          "Tackler1", "Tackler2", "PressureBy",
                          "InjuriesOnPlay", "InjuredPlayers",
                          "PenaltyOnPlay", "PenaltyPlayers"]]

,PlayDescription,PlayType,TimeOnTheClock,Formation,Quarterback,Receiver,PassType,PassDirection,PassYardage,Tackler1,Tackler2,PressureBy,InjuriesOnPlay,InjuredPlayers,PenaltyOnPlay,PenaltyPlayers
1,(11:39) (Shotgun) P.Mahomes pass short left to...,Pass,11:39,Shotgun,P.Mahomes,T.Kelce,Short,Left,1.0,C.Young,D.Greenlaw,nan,NaN,nan,NaN,nan
2,(11:04) (Shotgun) P.Mahomes pass short right t...,Pass,11:04,Shotgun,P.Mahomes,J.McKinnon,Short,Right,8.0,F.Warner,D.Greenlaw,nan,NaN,nan,NaN,nan
6,(5:15) (Shotgun) P.Mahomes pass short left to ...,Pass,5:15,Shotgun,P.Mahomes,R.Rice,Short,Left,0.0,F.Warner,nan,nan,NaN,nan,NaN,nan
10,(14:48) P.Mahomes pass short left to I.Pacheco...,Pass,14:48,nan,P.Mahomes,I.Pacheco,Short,Left,7.0,T.Gipson,nan,nan,NaN,nan,NaN,nan
13,(13:01) (Shotgun) P.Mahomes pass deep right to...,Pass,13:01,Shotgun,P.Mahomes,M.Hardman,Deep,Right,52.0,J.Brown,nan,nan,NaN,nan,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,(14:55) (Shotgun) B.Purdy pass short middle to...,Pass,14:55,Shotgun,B.Purdy,C.McCaffrey,Short,Middle,2.0,C.Jones,nan,nan,NaN,nan,NaN,nan
181,(12:38) (Shotgun) B.Purdy pass short left to B...,Pass,12:38,Shotgun,B.Purdy,B.Aiyuk,Short,Left,11.0,M.Edwards,nan,nan,NaN,nan,NaN,nan
183,(11:12) (Shotgun) B.Purdy pass short left to C...,Pass,11:12,Shotgun,B.Purdy,C.McCaffrey,Deep,Left,24.0,L.Sneed,nan,G.Karlaftis,NaN,nan,NaN,nan
186,(9:25) (Shotgun) B.Purdy pass short right to K...,Pass,9:25,Shotgun,B.Purdy,K.Juszczyk,Short,Right,13.0,nan,nan,nan,NaN,nan,NaN,nan


## Run

### Identify Different Run Plays
- This section is used to categorize different run plays to see if they have to be handled differently.
  - Eventually, each category of pass play will break down into the same set of features. The question here is how does each category of pass play break down to fall into these common features?

GOAL: To create a single method that will handle each type of pass play and break them down to a common set of features.

In [36]:
df_2023_run_sb['PlayOutcome'].unique()

array(['-3 Yard Run', '10 Yard Run', '4 Yard Run', '2 Yard Run',
       '5 Yard Run', '3 Yard Run', 'Run for No Gain', '6 Yard Run',
       '11 Yard Run', '-2 Yard Run', '24 Yard Run', '1 Yard Run',
       '-1 Yard Run', '8 Yard Run', '22 Yard Run', '9 Yard Run',
       '19 Yard Run', '-4 Yard Run', '7 Yard Run', '16 Yard Run'],
      dtype=object)

In [38]:
# 2 different formats ?

df_yard_runs = df_2023_run_sb[df_2023_run_sb['PlayOutcome'].str.contains('Yard Run')]
df_run_for_no_gain = df_2023_run_sb[df_2023_run_sb['PlayOutcome'].str.contains('Run for No Gain')]

In [40]:
df_yard_runs.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
0,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,2,KC,0,1,0,-3 Yard Run,(12:15) (Shotgun) I.Pacheco left guard to KC 2...,1st & 10 at KC 27
4,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,4,KC,0,1,0,10 Yard Run,(6:28) (Shotgun) I.Pacheco right guard to KC 2...,1st & 10 at KC 11
7,2023,Super Bowl,SUN,02/11,49ers,Chiefs,1ST QUARTER,4,KC,0,4,0,4 Yard Run,(4:36) (Shotgun) P.Mahomes scrambles up the mi...,3rd & 14 at KC 17
11,2023,Super Bowl,SUN,02/11,49ers,Chiefs,2ND QUARTER,2,KC,0,3,0,2 Yard Run,(14:15) (Shotgun) I.Pacheco right guard to KC ...,2nd & 3 at KC 32
12,2023,Super Bowl,SUN,02/11,49ers,Chiefs,2ND QUARTER,2,KC,0,4,0,5 Yard Run,(13:41) R.Rice right tackle to KC 37 for 3 yar...,3rd & 1 at KC 34


In [43]:
for idx, value in df_yard_runs['PlayOutcome'].items():
  play = df_2023_plays_sb['PlayDescription'].iloc[idx]
  print(play)
  print(value)
  names = re.findall(name_pattern, play)
  for i in names:
    print(i)

  print()

(12:15) (Shotgun) I.Pacheco left guard to KC 24 for -3 yards (N.Bosa, J.Hargrave).
-3 Yard Run
I.Pacheco
N.Bosa
J.Hargrave

(6:28) (Shotgun) I.Pacheco right guard to KC 21 for 10 yards (L.Ryan; J.Brown).
10 Yard Run
I.Pacheco
L.Ryan
J.Brown

(4:36) (Shotgun) P.Mahomes scrambles up the middle to KC 21 for 4 yards (R.Gregory).
4 Yard Run
P.Mahomes
R.Gregory

(14:15) (Shotgun) I.Pacheco right guard to KC 34 for 2 yards (K.Givens).
2 Yard Run
I.Pacheco
K.Givens

(13:41) R.Rice right tackle to KC 37 for 3 yards (L.Ryan; D.Greenlaw). FUMBLES (L.Ryan), recovered by KC-Ju.Watson at KC 37. Ju.Watson to KC 39 for 2 yards (J.Brown).
5 Yard Run
R.Rice
L.Ryan
D.Greenlaw
L.Ryan
Ju.Watson
Ju.Watson
J.Brown

(9:07) (Shotgun) I.Pacheco up the middle to KC 14 for 4 yards (F.Warner; J.Kinlaw).
4 Yard Run
I.Pacheco
F.Warner
J.Kinlaw

(8:35) (Shotgun) P.Mahomes scrambles right tackle to KC 17 for 3 yards (A.Armstead).
3 Yard Run
P.Mahomes
A.Armstead

(3:59) (Shotgun) I.Pacheco up the middle to KC 30 for 3 